In [ ]:
import numpy as np
import matplotlib.pyplot as plt


import uproot
import awkward as ak

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

In [ ]:
# Function to calculate weights analytically

def weight_fn(xx1, xx2, xx3, phi, costh):
    weight = 1. + xx1 * costh * costh + 2. * xx2 * costh * np.sqrt(1. - costh * costh) * np.cos(phi) + 0.5 * xx3 * (1. - costh * costh)* np.cos(2. * phi)
    return weight / (1. + costh * costh)

# Extracting Drell-Yan Angular Coefficients using Neural Network-based Classifiers with E906 LH2 Data

In this notebook, we use deep neural network-based classifiers to extract the DY angular coefficients. As the first step, let's define our classifier.

In [ ]:
class BMFClassifier(nn.Module):
    def __init__(self, input_dim: int = 8, output_dim: int = 1, hidden_dim: int = 64):
        super(BMFClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim, bias=True)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim, bias=True)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim, bias=True)
        self.bn3 = nn.BatchNorm1d(hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_dim, bias=True)
        self.bn4 = nn.BatchNorm1d(output_dim)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.bn1(self.fc1(x))
        x = self.relu(x)
        x = self.bn2(self.fc2(x))
        x = self.relu(x)
        x = self.bn3(self.fc3(x))
        x = self.relu(x)
        x = self.bn4(self.fc4(x))
        x = self.sigmoid(x)
        return x

    # Train step
    def train_model(self, criterion, optimizer, device, epochs, early_stopping_patience):

        # Load messy MC data
        tree = uproot.open("BMFData.root:save")
        
        data = tree.arrays(["true_phi", "true_costh", "mass", "pT", "xF", "phi", "costh"])
        
        # Number of MC samples
        n_MC_events = 10**6
        
        # Default values for lambda, mu, nu
        lambda0, mu0, nu0 = 1.0, 0.0, 0.0
        
        data0 = data[:n_MC_events]
        data1 = data[n_MC_events:2*n_MC_events]
        
        lambda_vals = np.random.uniform(0.5, 1.5, n_MC_events)
        mu_vals = np.random.uniform(-0.5, 0.5, n_MC_events)
        nu_vals = np.random.uniform(-0.5, 0.5, n_MC_events)
        
        X0 = [(mass, pT, xF, phi, costh, theta0, theta1, theta2) for mass, pT, xF, phi, costh, theta0, theta1, theta2 in zip(data0.mass, data0.pT, data0.xF, data0.phi, data0.costh, lambda_vals, mu_vals, nu_vals)]
        
        X1 = [(mass, pT, xF, phi, costh, theta0, theta1, theta2) for mass, pT, xF, phi, costh, theta0, theta1, theta2 in zip(data1.mass, data1.pT, data1.xF, data1.phi, data1.costh, lambda_vals, mu_vals, nu_vals)]
        
        Y0 = np.zeros(n_MC_events)
        Y1 = np.ones(n_MC_events)
        
        weight0 = [(weight_fn(lambda0, mu0, nu0, phi, costh)) for phi, costh in zip(data0.true_phi, data0.true_costh)]
        weight1 = [(weight_fn(theta0, theta1, theta2, phi, costh)) for theta0, theta1, theta2, phi, costh, in zip(lambda_vals, mu_vals, nu_vals, data1.true_phi, data1.true_costh)]
        
        X = np.concatenate((X0, X1))
        Y = np.concatenate((Y0, Y1)).reshape(-1, 1)
        weights = np.concatenate((weight0, weight1)).reshape(-1, 1)
        
        # Train test split
        X_train, X_test, Y_train, Y_test, weights_train, weights_test = train_test_split(X, Y, weights, test_size=0.3, shuffle=True)
        
        batch_size = 1024
        
        # Convert data to PyTorch tensors
        X_train_tensor = torch.from_numpy(X_train).float()
        Y_train_tensor = torch.from_numpy(Y_train).float()
        weights_train_tensor = torch.from_numpy(weights_train).float()
        
        X_test_tensor = torch.from_numpy(X_test).float()
        Y_test_tensor = torch.from_numpy(Y_test).float()
        weights_test_tensor = torch.from_numpy(weights_test).float()
        
        # Create PyTorch datasets and dataloaders
        train_dataset = TensorDataset(X_train_tensor, Y_train_tensor, weights_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, Y_test_tensor, weights_test_tensor)
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        
        best_loss = float('inf')
        best_model_weights = None
        patience_counter = 0
        
        # Train step
        for epoch in range(epochs):
            self.train()
            running_loss = 0.0
            for batch_inputs, batch_labels, batch_weights in train_loader:
                batch_inputs = batch_inputs.to(device)
                batch_labels = batch_labels.to(device)
                batch_weights = batch_weights.to(device)
                
                optimizer.zero_grad()
                outputs = self(batch_inputs)
                loss = criterion(outputs, batch_labels, batch_weights)
                
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * batch_inputs.size(0)
                
            epoch_loss = running_loss / len(train_loader.dataset)
            
            # Evaluation
            self.eval()
            with torch.no_grad():
                running_loss = 0.0
                for batch_inputs, batch_labels, batch_weights in test_loader:
                    batch_inputs = batch_inputs.to(device)
                    batch_labels = batch_labels.to(device)
                    batch_weights = batch_weights.to(device)
                    
                    outputs = self(batch_inputs)
                    loss = criterion(outputs, batch_labels, batch_weights)
                    
                    running_loss += loss.item() * batch_inputs.size(0)
                    
                validation_loss = running_loss / len(test_loader.dataset)
                
                # print("Epoch {}: Train Loss = {:.4f}, Test Loss = {:.4f}".format(epoch + 1, epoch_loss, validation_loss))
                
                # Check for early stopping
                if validation_loss < best_loss:
                    best_loss = validation_loss
                    best_model_weights = self.state_dict()
                    patience_counter = 0
                else:
                    patience_counter += 1
                    
                if patience_counter >= early_stopping_patience:
                    print("Early stopping at epoch {}".format(epoch))
                    break
        return best_model_weights

    # Reweight function
    def reweight_fn(self, X_val):
        # Move the model to CPU for evaluation
        model = self.to(torch.device("cpu"))
        
        model.eval()
        with torch.no_grad():
            preds = model(torch.Tensor(X_val)).detach().numpy().ravel()
            weights = preds / (1.0 - preds)
        
        return weights

    # Fit the model
    def fit_fn(self, epochs, add_params_layer, device, optimizer, loss_fn):
        # Load messy MC data
        tree = uproot.open("BMFData.root:save")
        
        data = tree.arrays(["true_phi", "true_costh", "mass", "pT", "xF", "phi", "costh"])
        
        # Number of MC samples
        n_MC_events = 10**6
        
        # Default values for lambda, mu, nu
        lambda0, mu0, nu0 = 1.0, 0.0, 0.0
        
        data0_0 = data[:n_MC_events]

        data0 = np.array([(mass, pT, xF, phi, costh, true_phi, true_costh) for mass, pT, xF, phi, costh, true_phi, true_costh in zip(data0_0.mass, data0_0.pT, data0_0.xF, data0_0.phi, data0_0.costh, data0_0.true_phi, data0_0.true_costh)])
        

        # Load real data
        tree1 = uproot.open("LH2Data.root:tree")

        data1_1 = tree1.arrays(["mass", "pT", "xF", "phi", "costh", "weight"])

        # Number of E906 events
        n_E906_events = data1_1.mass.to_numpy().shape[0]

        # Create validation data set
        data0_1, data0_2 = train_test_split(data0, test_size = n_E906_events/n_MC_events, shuffle=True)

        X0_val = np.array([(mass, pT, xF, phi, costh) for mass, pT, xF, phi, costh in zip(data0_2[:, 0], data0_2[:, 1], data0_2[:, 2], data0_2[:, 3], data0_2[:, 4])])
        
        X1_val = np.array([(mass, pT, xF, phi, costh) for mass, pT, xF, phi, costh in zip(data1_1.mass, data1_1.pT, data1_1.xF, data1_1.phi, data1_1.costh)])

        Y0_val = np.zeros(n_E906_events)
        Y1_val = np.ones(n_E906_events)

        weight0_val = [(weight_fn(lambda0, mu0, nu0, phi, costh)) for phi, costh in zip(data0_2[:, 5], data0_2[:, 6])]
        weight1_val = data1_1.weight.to_numpy()

        X = np.concatenate((X0_val, X1_val))
        Y = np.concatenate((Y0_val, Y1_val)).reshape(-1, 1)
        weights = np.concatenate((weight0_val, weight1_val)).reshape(-1, 1)

        # Define batch size
        batch_size = 1024
        
        # Create PyTorch datasets and dataloaders
        dataset = TensorDataset(torch.Tensor(X), torch.Tensor(Y).float(), torch.Tensor(weights))
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        
        losses = []
        fit_vals = {
            "lambda": [],
            "mu": [],
            "nu": []
        }

        
        for epoch in range(epochs):
            add_params_layer.train()
            running_loss = 0.0
            
            for batch_inputs, batch_labels, batch_weights in data_loader:
                batch_inputs = batch_inputs.to(device)
                batch_labels = batch_labels.to(device)
                batch_weights = batch_weights.to(device)
                
                # Forward pass
                optimizer.zero_grad()
                param_input = add_params_layer(batch_inputs)
                output = self(param_input)
                
                # Compute the loss
                loss = loss_fn(output, batch_labels, batch_weights)
                
                # Backward pass and update weights
                loss.backward()
                optimizer.step()
                
                running_loss += loss.item() * batch_inputs.size(0)
                
            epoch_loss = running_loss / len(data_loader.dataset)
            # print("epoch : {}, loss = {:.4f}, lambda = {:.4f}, mu = {:.4f}, nu = {:.4f}".format(epoch + 1, epoch_loss,
            #                                                                                 add_params_layer.params[0].item(),
            #                                                                                 add_params_layer.params[1].item(),
            #                                                                                 add_params_layer.params[2].item()))
            losses.append(epoch_loss)
            fit_vals["lambda"].append(add_params_layer.params[0].item())
            fit_vals["mu"].append(add_params_layer.params[1].item())
            fit_vals["nu"].append(add_params_layer.params[2].item())
            
        return fit_vals

We extract the angular coefficients in two steps:

## Step 1: Parameterize the neural network with angular coefficients

In this step, we parameterize the neural network with $\lambda$, $\mu$, $\nu$ values. This is done during the training step. The input features to the neural network are `mass`, `pT`, `xF`, `phi`, `costh`, `lambda`, `mu`, and `nu`.

## Step 2: Extract the parameters using the gradient descent algorithm

Since we have parameterized the neural network in step 1, we can fix the trained weights in the neural network and extract the angular coefficients by minimizing the loss with the gradient descent algorithm.

Let's select the `cuda` device for faster training.

In [ ]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

Let's define a custom loss function to classify the classes with weights.

In [ ]:
# Custom loss function
class BMFLoss(nn.Module):
    def __init__(self):
        super(BMFLoss, self).__init__()
        
    def forward(self, outputs, targets, weights):
        weighted_targets = targets * weights + (1 - targets) * (1 - weights)
        criterion = nn.BCELoss()
        loss = criterion(outputs, weighted_targets)
        return loss

Let's define a parameter extraction layer.

In [ ]:
# Module used to add parameter for fitting
class AddParams2Input(nn.Module):
    def __init__(self, params):
        super(AddParams2Input, self).__init__()
        self.params = nn.Parameter(torch.Tensor(params), requires_grad=True)

    def forward(self, inputs):
        batch_params = torch.ones((inputs.size(0), 1), device=inputs.device) * self.params.to(device=inputs.device)
        concatenated = torch.cat([inputs, batch_params], dim=-1)
        return concatenated

Let's define our model.

We use the `Adam` optimizer for backpropagation.

In [ ]:
# Define number of epochs
epochs = 200

# Define early stopping patience
early_stopping_patience = 20

# Define number of iterations
iterations = 20

## Fit Model to Extract the Parameters

In [ ]:
lambda_fit, mu_fit, nu_fit = [], [], []

for i in range(iterations):

    print("iteration = {}".format(i+1))

    fit_model = BMFClassifier(input_dim=8, hidden_dim=64)

    # Define the loss function and optimizer
    criterion = BMFLoss()
    optimizer = optim.Adam(fit_model.parameters(), lr=0.001)
    
    # Move the model to GPU if available
    fit_model = fit_model.to(device=device)

    # Compile the model
    opt_model = torch.compile(fit_model)

    # Model summary
    # print("using device : {}".format(device))
    # total_trainable_params = sum(p.numel() for p in fit_model.parameters() if p.requires_grad)
    # print(fit_model)
    # print('total trainable params: {}'.format(total_trainable_params))

    # Train the model
    best_model_weights = opt_model.train_model(criterion, optimizer, device, epochs, early_stopping_patience)

    # Load the best model weights
    fit_model.load_state_dict(best_model_weights)

    # Initialize the parameters
    fit_init = [np.random.uniform(0.5, 1.5, 1)[0], np.random.uniform(-0.5, 0.5, 1)[0], np.random.uniform(-0.5, 0.5, 1)[0]]

    # Create the AddParams2Input layer
    add_params_layer = AddParams2Input(fit_init)

    # Set all weights in fit model to non-trainable
    for param in fit_model.parameters():
        param.requires_grad = False

    # Define the loss function and optimizer
    loss_fn = BMFLoss()
    optimizer = torch.optim.Adam(add_params_layer.parameters(), lr=0.001)

    # Transfer models to GPU
    add_params_layer = add_params_layer.to(device)
    fit_model = fit_model.to(device)

    # Model summary
    # print("using device : {}".format(device))
    # fit_trainable_params = sum(p.numel() for p in fit_model.parameters() if p.requires_grad)
    # print(fit_model)
    # print("total trainable params in fit model: {}".format(fit_trainable_params))

    # total_trainable_params = sum(p.numel() for p in add_params_layer.parameters() if p.requires_grad)
    # print(add_params_layer)
    # print("total trainable params in fit model: {}".format(total_trainable_params))

    fit_vals = fit_model.fit_fn(epochs, add_params_layer, device, optimizer, loss_fn)
    
    lambda_fit.append(fit_vals["lambda"][-1])
    mu_fit.append(fit_vals["mu"][-1])
    nu_fit.append(fit_vals["nu"][-1])

In [ ]:
print("lambda fit = {:.4f} +/- {:.4f}".format(np.mean(lambda_fit), np.std(lambda_fit)))
print("mu fit = {:.4f} +/- {:.4f}".format(np.mean(mu_fit), np.std(mu_fit)))
print("nu fit = {:.4f} +/- {:.4f}".format(np.mean(nu_fit), np.std(nu_fit)))